# On modelling optimization problems via Julia JuMP

## Prof. Mayron César O. Moreira 

**Universidade Federal de Lavras (UFLA)**  
**Department of Computer Science**  
**Lavras, Minas Gerais, Brazil**  

*Università degli Studi di Modena e Reggio Emilia (UNIMORE)*  
*Reggio Emilia, Italy*

## Final Test

In groups of $x$ students, where $2 \le x \le 3$, solve the following problems using Julia and JuMP.

Since the group finishes, it must send the notebook with the answers to mayron.moreira@ufla.br **and** Manuel.iori@unimore.it.

One member of the group will send the e-mail, and in the body of the message, indicates that students that make part of your group.

### Problem 1

Moe is deciding how many Regular Duffs and Special Duffs he needs to deliver in each week. The Regular Duff costs to Moe \\$1 for each bottle, and he sells it by \\$2. Special Duff costs \\$1.5 for each bottle, and he sells it by \\$3. However, as part of a controversial marketing strategy, the Duff company only sells one Special Duff bottle for at least two bottles of Regular Duff that Moe buys. Moreover, due to some problems happened some years ago, Duff will not sell to Moe more than 3.000 bottles by week. According to his experience, Moe knows that he can sell all the beers that we can buy. Now, it is your turn: **implement a mathematical model** using Julia and JuMP, that helps Moe to decide how many Regular Duffs and Special Duffs he must buy to optimize his profit.

In [6]:
using JuMP, Clp
model = Model(with_optimizer(Clp.Optimizer))

@variable(model, x >= 0) # Regulas Duffs
@variable(model, y >= 0) # Special Duffs

@objective(model, Max, x + 1.5y)

@constraint(model, x + y <= 3000)
@constraint(model, 2*y <= x)

optimize!(model)
println(objective_value(model))

3500.0
Coin0506I Presolve 0 (-2) rows, 0 (-2) columns and 0 (-4) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value 3500
Coin0511I After Postsolve, objective 3500, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 3500 - 0 iterations time 0.002, Presolve 0.00


### Problem 2

Let $V = \{1,..,n\}$ be a set of $n$ objects, each one with a specific weight $w_i$, $i \in N$. Suppose that we have $n$ boxes available, each one with a capacity $C$. Write a mathematical model in Julia + JuMP that minimizes the number of boxes we will use, guaranteeing that all items are placed in a box and the used boxes do not have their capacity violated.

**Hint**: look for *the bin packing problem*. As follows, we have the parameters that must be used in the mathematical formulation.

In [10]:
n = 20
V = collect(1:n)
B = collect(1:n)
C = 50

w = [9, 36, 36, 17, 23, 29, 23, 50, 36, 38, 33, 43, 49, 6, 31, 30, 11, 8, 21, 49]
println("Set of itens: ", V)
println("Set of boxes: ", B)
println("Weights: ", w)

using Cbc
model = Model(with_optimizer(Cbc.Optimizer))

@variable(model, x[V,B], Bin)
@variable(model, y[V], Bin)

@objective(model, Min, sum(y[j] for j in B))

@constraint(model, [i in V], sum(x[i,j] for j in B) == 1)
@constraint(model, [j in B], sum(w[i]*x[i,j] for i in V) <= C*y[j])
@constraint(model, [j=2:n], y[j] <= y[j-1])

optimize!(model)

println(objective_value(model))

Set of itens: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Set of boxes: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Weights: [9, 36, 36, 17, 23, 29, 23, 50, 36, 38, 33, 43, 49, 6, 31, 30, 11, 8, 21, 49]
Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 11.56 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 9 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 8 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 9 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 9 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 8 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tighte

In [45]:
println("Write your code here!")

Write your code here!


### Problem 3

* **Exercise extracted from https://github.com/pedrocastellucci/MathematicalProgrammingCourse/blob/master/examples/Example%2001%20-%20The%20travelling%20salesman%20meets%20Pokemon%20Go.ipynb**

[Pokemon Go](https://www.pokemongo.com/it-it/) is a game developed by [Niantic](https://en.wikipedia.org/wiki/Pok%C3%A9mon) which consists of collecting items (or Pokemons) by players through specific placed called Pokestops. Suppose that we have the following Pokestops, represented by their coordinates below.

A player wants to leave the Pokestop one, visit all of them and go back to its starting point, with a minimum walking distance. Implement a code in Julia that models and solves this problem.

Another way to model this problem is to use the *One-Commodity Formulation*. Let´s implement it! For this purpose, let $x_{ij}$ be a binary variable equal to 1 if we travel by arc $(i,j)$ in the Hamiltonian cycle. We define $f_{ij} \ge 0$ as the flow in the arc $(i,j)$. Then, the model reads.

\begin{equation}
\min \sum_{i=1}^n \sum_{j=1}^n c_{ij}x_{ij}
\end{equation}

subject to:

\begin{equation}
\sum_{i=1,i\neq j}^n x_{ij} = 1, \quad j=1,...,n
\end{equation}

\begin{equation}
\sum_{j=1,i\neq j}^n x_{ij} = 1, \quad i=1,...,n
\end{equation}

\begin{equation}
f_{ij} \le (n-1)x_{ij} \quad i,j=1,...,n
\end{equation}

\begin{equation}
\sum_{j=1}^n f_{1j} = n-1
\end{equation}

\begin{equation}
\sum_{i=1}^n f_{ij} = \sum_{k=1}^n f_{jk} + 1 \quad j=2,...,n
\end{equation}

\begin{equation}
x_{ij} \in \{0,1\} \quad i,j=1,...,n
\end{equation}

\begin{equation}
0 \le f_{ij} \le n-1 \quad i=1,...,n
\end{equation}

Using the following data, implement a code in Julia + JuMP that solves this problem.

In [46]:
Pokestops = Dict(
    1 => (523, 418),
    2 => (527, 566),
    3 => (435, 603),
    4 => (386, 660),
    5 => (346, 692),
    6 => (431, 730),
    7 => (419, 818),
    8 => (389, 892),
    9 => (384, 902),
    10 => (383, 913),
    11 => (347, 520),
    12 => (332, 330),
    13 => (165, 374),
    14 => (196, 198),
    15 => (187, 108),
    16 => (210, 63)
)
;

In [47]:
# Number of points
n = 16

# Euclidian distance
EucD(x,y) = sqrt((x[1] - y[1])^2 + (x[2] - y[2])^2)

# Cost matrix
c = [EucD(Pokestops[i],Pokestops[j]) for i=1:n, j=1:n]
;

Write your JuMP model below.

In [48]:
println("Write here!")
println("Set your model to run at most 180s, with 1 single thread.")

Write here!
Set your model to run at most 180s, with 1 single thread.
